# プロンプトエンジニアリング

プロンプトエンジニアリングでは、モデル呼び出し以前に入力設計と評価設計を固める習慣を作ります。


## 概念の土台

プロンプトエンジニアリングに入る前に、つまずきやすい用語を先にそろえます。以降のコードでは、変数がどの概念を表しているかを対応付けながら読んでください。

- **トークン**: モデルが処理する最小単位です。日本語では1文字単位とは限りません。
- **次トークン確率**: 文脈に対して次に出る候補の確率分布です。生成の中核です。
- **コンテキスト**: モデルに渡す入力全体です。指示・資料・履歴が含まれます。
- **指示・制約・出力形式**: 目的と禁止事項と返答形を分離して指定する設計です。
- **評価観点**: 良い応答の基準を先に決めることで改善ループが回ります。

このノートでは、ここで定義した語を実験セルの変数・式に直接対応させて確認します。


## 検証 1: トークン近似を体験する

最初に、入力長とトークン量の関係を簡易計測します。コスト管理の第一歩です。


In [ ]:
text = '大規模言語モデルは文脈の与え方で応答品質が大きく変わる。'
char_len = len(text)
space_tokens = text.split()
rough_tokens = max(1, char_len // 2)
print('chars=', char_len, 'space_tokens=', len(space_tokens), 'rough_tokens=', rough_tokens)

厳密なトークン化ではありませんが、入力を短く保つ設計感覚を作るには十分です。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 検証 2: 制約付きプロンプト設計

プロンプトは長く書くより、制約を明確化する方が効果的です。ここではテンプレート化を練習します。


In [ ]:
task = '連鎖律を初学者向けに説明する'
rules = ['120字以内', '式は1つまで', '最後に確認問題を1問']
template = f"課題: {task}\n制約: {'; '.join(rules)}\n出力形式: 段落1つ"
print(template)
print('template_len=', len(template))

この設計は生成APIを呼ばなくても、入力仕様の品質点検として価値があります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 定義の確認

1. $p_{\theta}(x_t \mid x_{<t})$
2. $L_{CE} = -\sum_t \log p_{\theta}(x_t \mid x_{<t})$


## 検証 3: 検索文脈を結合する

ここで RAG の最小形を実装します。質問と関連文を結合し、回答入力を作る流れを確認します。


In [ ]:
question = 'ベルマン方程式を高校生向けに説明して'
retrieved = ['価値は将来報酬の割引和で定義する', '現在価値は次状態価値で再帰的に更新できる']
context = '\n'.join(f'- {c}' for c in retrieved)
final_input = f"質問:\n{question}\n\n参考文脈:\n{context}\n\n回答:"
print(final_input)

検索文脈を入れる目的は、モデルの記憶に頼りすぎないことです。根拠付き応答を作りやすくなります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 検証 4: 評価項目を数値化する

次に、回答を点検するための簡易スコアを定義します。評価軸を言語化すると改善が継続できます。


In [ ]:
answer = 'ベルマン方程式は、今の価値を次の価値で更新する再帰式です。'
checks = {'length_ok': len(answer) <= 120, 'has_keyword': '価値' in answer, 'has_recurrence': '再帰' in answer}
score = sum(1 for v in checks.values() if v) / len(checks)
print('checks=', checks)
print('score=', round(score, 3))

このような軽量評価でも、改善方向を揃える効果があります。実務ではこの評価軸をチームで共有します。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 検証 5: 推論コストを見積もる

最後に、入力と出力の長さから概算コストを計算します。モデル選択は性能だけでなく費用とのバランスが必要です。


In [ ]:
input_tokens = 320
output_tokens = 180
price_per_1k = 0.0012
cost = (input_tokens + output_tokens) / 1000 * price_per_1k
print('estimated_cost=', round(cost, 6))

この見積もりを運用前に作ると、スケール時の予算超過を防ぎやすくなります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## まとめ

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. プロンプトだけで全問題を解決しようとする
2. 評価指標を決めずに改善を繰り返す
3. コストと品質のバランスを見ない
